In [ ]:
'''
CLV анализ
'''

In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import zipfile
import io

# размер дохода маржи.

profit_margin = 0.10

In [2]:
archive = zipfile.ZipFile('online_retail_II.csv.zip', 'r')
txtdata = archive.read('online_retail_II.csv')
transaction_list = pd.read_csv(io.BytesIO(txtdata), encoding='cp1251')
transaction_list.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [3]:
transaction_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


In [4]:
# Буква C в счете-фактуре указывает на возврат.
# Мы пропускаем возвраты в нашей работе и оцениваем только стандартные счета-фактуры.

transaction_list = transaction_list[~transaction_list["Invoice"].str.contains("C")]

In [5]:
# статистика числовых переменных

transaction_list.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,1047877.0,10.592354,135.280603,-9600.00,1.00,3.0,10.00,80995.00
Price,1047877.0,3.901109,94.357740,-53594.36,1.25,2.1,4.13,25111.09
Customer ID,805620.0,15331.856250,1696.768395,12346.00,13982.00,15271.0,16805.00,18287.00


In [6]:
# Опустим значения ниже 1.

transaction_list = transaction_list[transaction_list["Quantity"] > 0] 
transaction_list = transaction_list[transaction_list["Price"] > 0]

In [7]:
# Для переменной Quantity значение 99-го процентиля равно 108, но максимальное значение равно 80995.
# Это может вызвать проблемы, поэтому я отбросил значения выше 99-го процентиля для переменных Quantity и Price

upper_limit_q = transaction_list["Quantity"].quantile(0.99)
upper_limit_p = transaction_list["Price"].quantile(0.99)  
transaction_list = transaction_list[transaction_list["Quantity"] <= upper_limit_q]
transaction_list = transaction_list[transaction_list["Price"] <= upper_limit_p]

In [8]:
# Изменение типа данных InvoiceDate на datetime

transaction_list["InvoiceDate"] = pd.to_datetime(transaction_list["InvoiceDate"])

# Для выполнения нашей работы данные должны быть сгруппированы по идентификатору клиента.
# Нам также необходимо общее количество счетов-фактур для каждого клиента,
# дата последнего счета-фактуры и общая стоимость для каждого клиента.

clv_df = transaction_list.groupby('Customer ID').agg({'Invoice': lambda x: x.nunique(),
                                        'Quantity': lambda x: x.sum(),
                                        'Price': lambda x: x.sum()})
clv_df.columns = ['total_transaction', 'total_unit', 'total_price']

# Основные формулы.

# формула 1: общая цена/общая сумма транзакции

clv_df["average_order_value"] = clv_df["total_price"] / clv_df["total_transaction"]

# формула 2: общая_транзакция / общее_количество_клиентов

clv_df["purchase_frequency"] = clv_df["total_transaction"] / clv_df.shape[0]

# формула 3: Коэффициент оттока - (1-((количество клиентов, сделавших заказ более одного раза / общее количество клиентов)))

churn_rate = 1- clv_df[clv_df["total_transaction"] > 1].shape[0] / clv_df.shape[0]

# формула 4: Маржа прибыли для каждого клиента

clv_df['profit_margin'] = clv_df['total_price'] * profit_margin

# формула 5: Ценность клиента (ценность_клиента = средняя_стоимость_заказа * частота_покупок)

clv_df['customer_value'] = clv_df['average_order_value'] * clv_df["purchase_frequency"]

# # Окончательная формула: CLV = (ценность_клиента / отток_клиентов) x profit_margin

clv_df["clv"] = (clv_df["customer_value"] / churn_rate) * clv_df["profit_margin"]

In [9]:
# Т.к. мы рассчитываем Customer Lifetime Value для каждого клиента, пришло время разделить их на сегменты.
# Разделения на 5 сегментов будет достаточно.

clv_df["segment"] = pd.qcut(clv_df["clv"], 5, labels=["E", "D", "C", "B", "A"])

# Рассмотрим средние и общие значения для каждого сегмента.

customer_segments = clv_df.groupby("segment").agg({"count", "mean", "sum"})
customer_segments

total_transaction                   total_unit                        \
                    count    sum       mean      count      sum         mean   
segment                                                                        
E                    1160   1768   1.524138       1160   208054   179.356897   
D                    1159   2513   2.168248       1159   368720   318.136324   
C                    1160   4084   3.520690       1160   725447   625.385345   
B                    1159   7051   6.083693       1159  1364602  1177.396031   
A                    1160  20359  17.550862       1160  4819999  4155.171552   

        total_price                           average_order_value  ...  \
              count          sum         mean               count  ...   
segment                                                            ...   
E              1160    28300.381    24.396880                1160  ...   
D              1159    89239.590    76.997058                1159  ...   
C              1160   187082.934   161.278391                1160  ...   
B              1159   396341.105   341.968167                1159  ...   
A              1160  1574634.808  1357.443800                1160  ...   

        purchase_frequency profit_margin                           \
                      mean         count          sum        mean   
segment                                                             
E                 0.000263          1160    2830.0381    2.439688   
D                 0.000374          1159    8923.9590    7.699706   
C                 0.000607          1160   18708.2934   16.127839   
B                 0.001049          1159   39634.1105   34.196817   
A                 0.003027          1160  157463.4808  135.744380   

        customer_value                         clv                             
                 count         sum      mean count            sum        mean  
segment                                                                        
E                 1160    4.881059  0.004208  1160      56.363524    0.048589  
D                 1159   15.391444  0.013280  1159     449.484846    0.387821  
C                 1160   32.266805  0.027816  1160    1954.126217    1.684592  
B                 1159   68.358245  0.058980  1159    8947.488173    7.720007  
A                 1160  271.582409  0.234123  1160  416510.141091  359.060466  

[5 rows x 24 columns]

In [10]:
# Импортируем необходимые библиотеки

!pip install lifetimes
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions
import datetime as dt

In [11]:
# В этом методе используются данные о дате выставления счета.
# Подобно метрике недавности для анализа RFM, она используется для оценки наиболее частых транзакций каждого клиента.

# В наших данных наиболее частый счет-фактура датирован 09.12.2011.
# Поэтому мы будем использовать следующий день как самую позднюю дату.

most_recent_date = dt.datetime(2011, 12, 10)

# аналогично первому методу, мы создадим новый датафрейм с агрегацией данных каждого клиента.

clv_p_df = transaction_list.groupby('Customer ID').agg(
    {'InvoiceDate': [lambda InvoiceDate: (InvoiceDate.max() - InvoiceDate.min()).days,
                     lambda InvoiceDate: (most_recent_date - InvoiceDate.min()).days],
     'Invoice': lambda Invoice: Invoice.nunique(),
     'Price': lambda Price: Price.sum()})

# переименование столбцов

clv_p_df.columns = ['recency', 'T', 'frequency', 'monetary']

# расчет средней стоимости транзакции

clv_p_df["monetary"] = clv_p_df["monetary"] / clv_p_df["frequency"]   

# в наших расчетах, чтобы оценить CLV для клиентов, клиент должен совершить не менее 2 покупок.

clv_p_df = clv_p_df[(clv_p_df['frequency'] > 1)] 

# Переведем наши временные данные в недельную основу.

clv_p_df["recency"] = clv_p_df["recency"] / 7 
clv_p_df["T"] = clv_p_df["T"] / 7

# BG/NBD модель

bgf = BetaGeoFitter(penalizer_coef=0.001)

bgf.fit(clv_p_df['frequency'],
        clv_p_df['recency'],
        clv_p_df['T'])

# Gamma-Gamma модель

ggf = GammaGammaFitter(penalizer_coef=0.01)

ggf.fit(clv_p_df['frequency'], clv_p_df['monetary'])

ggf.conditional_expected_average_profit(clv_p_df['frequency'],
                                        clv_p_df['monetary']).head(10)

ggf.conditional_expected_average_profit(clv_p_df['frequency'],
                                        clv_p_df['monetary']).sort_values(ascending=False).head(10)

clv_p_df["expected_average_profit"] = ggf.conditional_expected_average_profit(clv_p_df['frequency'],
                                                                             clv_p_df['monetary'])

# Найдем CLV с помощью BG-NBD и Gamma Gamma моделей

clv_p = ggf.customer_lifetime_value(bgf,
                                   clv_p_df['frequency'],
                                   clv_p_df['recency'],
                                   clv_p_df['T'],
                                   clv_p_df['monetary'],
                                   time=3,  # 3 aylık
                                   freq="W",  # T'nin frekans bilgisi.
                                   discount_rate=0.01)

# Последние штрихи к модели и объединение ее с нашим первоначальным df

clv_p = clv_p.reset_index()
clv_final = clv_p_df.merge(clv_p, on="Customer ID", how="left")

# Подобно первому методу, мы можем получить различные сегменты с помощью функции qcut

clv_final["segment"] = pd.qcut(clv_final["clv"], 4, labels=["D", "C", "B", "A"])

# Получение среднего и суммарного значений для каждого сегмента

clv_final.groupby("segment").agg(
    {"count", "mean", "sum"})

Customer ID                           recency                \
              count         sum          mean   count           sum   
segment                                                               
D              1048  16091813.0  15354.783397    1048  31594.428571   
C              1047  16096415.0  15373.844317    1047  60286.857143   
B              1047  15970975.0  15254.035339    1047  67444.000000   
A              1047  15986980.0  15269.321872    1047  66782.428571   

                       T                          frequency  ...             \
              mean count           sum       mean     count  ...       mean   
segment                                                      ...              
D        30.147356  1048  84126.000000  80.272901      1048  ...   4.207061   
C        57.580570  1047  79462.285714  75.895211      1047  ...   4.862464   
B        64.416428  1047  76369.142857  72.940920      1047  ...   7.574976   
A        63.784555  1047  70954.000000  67.768863      1047  ...  15.983763   

        monetary                            expected_average_profit  \
           count            sum        mean                   count   
segment                                                               
D           1048   42160.460123   40.229447                    1048   
C           1047   48878.872924   46.684692                    1047   
B           1047   69266.540265   66.157154                    1047   
A           1047  108789.250350  103.905683                    1047   

                                     clv                             
                   sum        mean count            sum        mean  
segment                                                              
D         44989.253159   42.928677  1048    5570.382426    5.315250  
C         51510.577896   49.198260  1047   31120.405729   29.723406  
B         71715.586335   68.496262  1047   74448.884597   71.106862  
A        111301.485988  106.305144  1047  247209.168466  236.111909  

[4 rows x 21 columns]

In [ ]:
'''
Полученные значения отличаются от простого разделения с помощью qcut().
Количество значений чуть меньше, но средние и суммарные значения выше.
Необходимо рассматривать значиния в отдельных случаях, но я склонююсь ко второму методу, с BG/NBD моделью и Gamma Gamma моделью.
'''